<a href="https://colab.research.google.com/github/fatemafaria142/Comparative-Analysis-of-Diverse-Large-Language-Models-in-Story-Generation/blob/main/Story_Generation_using_Llama_2_7b_chat_hf_and_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━

#### **Dataset Link:** https://huggingface.co/datasets/AtlasUnified/atlas-storyteller

In [2]:
from datasets import load_dataset

# Load the dataset
instruct_tune_dataset = load_dataset("AtlasUnified/atlas-storyteller")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
# Select 500 rows for training
train_subset = instruct_tune_dataset["train"].select(indices=range(500))

# Select 200 rows for testing
test_subset = instruct_tune_dataset["train"].select(indices=range(500, 700))


In [4]:
train_subset

Dataset({
    features: ['id', 'Story'],
    num_rows: 500
})

In [5]:
test_subset

Dataset({
    features: ['id', 'Story'],
    num_rows: 200
})

In [6]:
# Check the sizes of the subsets
print("Train subset size:", len(train_subset))
print("Test subset size:", len(test_subset))

Train subset size: 500
Test subset size: 200


In [8]:
# Show 3 examples from the train dataset
print("Train subset examples:")
for i in range(3):
    id = train_subset['id'][i]
    story = train_subset['Story'][i]
    print(f"Example {i + 1}:")
    print(f"ID: {id}")
    print(f"Story: {story}")
    print("-------------------")

Train subset examples:
Example 1:
ID: seed_task_0
Story: In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They're coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why were they after him? Ethan's blood ran cold, fueling a surge of adrenaline that urged him to take action.  Without a moment's hesitation, Ethan gathered his meager belongings and made his way to a hidden room beneath his

In [9]:
# Show 3 examples from the test dataset
print("Test subset examples:")
for i in range(3):
    id = test_subset['id'][i]
    story = test_subset['Story'][i]
    print(f"Example {i + 1}:")
    print(f"ID: {id}")
    print(f"Story: {story}")
    print("-------------------")

Test subset examples:
Example 1:
ID: seed_task_500
Story: In the heart of Boston sits the grand Isabella Stewart Gardner Museum, an oasis of art and culture. With its ornate architecture and lush gardens, the museum is a testament to beauty and history. Within its walls resides a breathtaking collection of masterpieces by the greatest artists of all time.  It was a cool spring evening when two men, Mark and Ethan, stood outside the museum, their eyes fixed on the magnificent building that lay before them. They were not ordinary visitors or enthusiasts; they were art thieves plotting a daring heist. The allure of the Gardner Museum's valuable treasures had drawn them in like moths to a flame.  Mark, a seasoned thief with an eye for detail, had spent months carefully studying the museum's layout, security systems, and the positioning of the most prized artworks. Ethan, his young and eager apprentice, had admired Mark's skills and had accepted the opportunity to learn from a master.  They

#### **Creating a Mental Health Support Prompt Using LangChain**

In [25]:
from langchain import PromptTemplate, LLMChain

def create_prompt(sample):
    # Creating prompt for large language model
    pre_prompt = """[INST] <<SYS>>\nImagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story!\n"""
    prompt = pre_prompt + "{Story}[\INST]"

    # Extract Story from the sample
    example_response = sample['Story']

    # Create a prompt using the template and the example
    prompt_template = PromptTemplate(template=prompt, input_variables=['Story'])
    single_turn_prompt = prompt_template.format(Story=example_response)

    return single_turn_prompt


#### **Generating LangChain Prompt for a Training Subset Examples**

In [26]:
prompt = create_prompt(train_subset[0])
print(prompt)

[INST] <<SYS>>
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story!
In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They're coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why were th

In [27]:
prompt = create_prompt(train_subset[1])
print(prompt)

[INST] <<SYS>>
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story!
The sky stretched out before them, an endless expanse of blue pierced only by fluffy white clouds. The air crackled with the anticipation of action as the squadron gathered at the airfield, their sleek fighter jets gleaming under the noon sun. Among the pilots was Captain Joshua Bennett, a seasoned veteran with eyes as sharp as a hawk's, and a heart that beat with the rhythm of war.  Set in the midst of a raging conflict, the story takes place during the height of World War II. Captain Bennett, a fearless and daring leader, had earned his reputation through countless successful missions. He was known for his audacious maneuvers and masterful control of aircraft, traits that instilled confidence in every member of his squ

In [28]:
prompt = create_prompt(train_subset[2])
print(prompt)

[INST] <<SYS>>
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story!
In the midst of a bustling metropolis teeming with skyscrapers that seemingly touched the heavens, nestled a small and unassuming garage. It belonged to Ethan, a young and brilliant engineer consumed by his insatiable thirst for innovation and adventure. With grease-stained hands and a mind full of ideas, he spent every waking hour tinkering and crafting marvelous inventions that could turn dreams into reality.  One fateful afternoon, while rummaging through a box filled with ancient blueprints, Ethan stumbled upon a peculiar schematic. It depicted a colossal machine, a menacing giant robot with gleaming steel limbs and an aura of unstoppable power. Intrigued and fascinated, he couldn't resist the temptation to bring thi

#### **Generating LangChain Prompt for a Testing Subset Examples**

In [29]:
prompt = create_prompt(test_subset[0])
print(prompt)

[INST] <<SYS>>
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story!
In the heart of Boston sits the grand Isabella Stewart Gardner Museum, an oasis of art and culture. With its ornate architecture and lush gardens, the museum is a testament to beauty and history. Within its walls resides a breathtaking collection of masterpieces by the greatest artists of all time.  It was a cool spring evening when two men, Mark and Ethan, stood outside the museum, their eyes fixed on the magnificent building that lay before them. They were not ordinary visitors or enthusiasts; they were art thieves plotting a daring heist. The allure of the Gardner Museum's valuable treasures had drawn them in like moths to a flame.  Mark, a seasoned thief with an eye for detail, had spent months carefully studying the

In [30]:
prompt = create_prompt(test_subset[1])
print(prompt)

[INST] <<SYS>>
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story!
In the hushed halls of the grand Belvedere Palace, where a myriad of masterpieces adorned the walls, stood the renowned painting—Gustav Klimt's "The Kiss." Its exquisite depiction of two lovers locked in an eternal embrace captivated the hearts and minds of all who beheld it. Immersed in its artistic brilliance, visitors from near and far flocked to the museum, eager to witness this iconic masterpiece firsthand. But unbeknownst to the curator and the enchanted crowd, a nefarious plot was about to unfold.  Deep within the shadows of the city, a notorious art thief named Raphael Thorne conceived a daring plan to steal "The Kiss." Known for his audacious heists, Thorne had successfully pilfered priceless works of art from t

In [31]:
prompt = create_prompt(test_subset[2])
print(prompt)

[INST] <<SYS>>
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story!
The gallery was dimly lit, a hushed silence enveloping the room as visitors moved reverently from one masterpiece to another. Nestled within the heart of the city, the prestigious Vermeulen Gallery stood as a bastion of artistic brilliance, housing an impressive collection of priceless works. Amongst those works were several paintings, steeped in history and tragedy, stolen from Holocaust victims during the dark days of World War II. These stolen masterpieces whispered tales of sorrow and loss, waiting to be returned to their rightful owners.  Unknown to the gallery's patrons, a shadowy figure named Thomas 'The Phantom' Wagner had long harbored a plan to liberate the stolen art and restore justice. A master connoisseur a

In [32]:
prompt = create_prompt(test_subset[2])
print(prompt)

[INST] <<SYS>>
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story!
The gallery was dimly lit, a hushed silence enveloping the room as visitors moved reverently from one masterpiece to another. Nestled within the heart of the city, the prestigious Vermeulen Gallery stood as a bastion of artistic brilliance, housing an impressive collection of priceless works. Amongst those works were several paintings, steeped in history and tragedy, stolen from Holocaust victims during the dark days of World War II. These stolen masterpieces whispered tales of sorrow and loss, waiting to be returned to their rightful owners.  Unknown to the gallery's patrons, a shadowy figure named Thomas 'The Phantom' Wagner had long harbored a plan to liberate the stolen art and restore justice. A master connoisseur a

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [34]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

* **Model and Tokenizer:** https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

In [35]:
mode_id = "NousResearch/Llama-2-7b-chat-hf"

In [36]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [37]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently  (without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [39]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_p = 0.7  # You can adjust this value based on your preference
temperature = 0.9  # You can adjust this value based on your preference

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, do_sample=do_sample, pad_token_id=pad_token_id,  top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipe)

# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nImagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story!\n"""
prompt = pre_prompt + "{Story}[\INST]"

prompt = PromptTemplate(template=prompt, input_variables=["Story"])

chain = prompt | llm

Story = "In Dhaka's vibrant Ahsanullah University, Asif Fahim, Mukaffi, and Morshed forged a lasting bond. As students, they navigated the complexities of science and technology, each with unique dreams."

print(chain.invoke({"Story": Story}))

  As the days passed, the trio's friendship grew stronger, fueled by their shared passion for science and technology. They spent countless hours discussing their research projects, debating the latest scientific breakthroughs, and supporting each other through the challenges of university life.

One day, while wandering through the campus, they stumbled upon an old, mysterious-looking book hidden away in the university library's dusty archives. The cover was worn and faded, but the pages were filled with strange symbols and diagrams that seemed to defy explanation.

Intrigued, the trio decided to investigate further. They spent hours poring over the book, trying to decipher its secrets. As they delved deeper into the mysteries of the book, they began to notice strange occurrences happening around them. Equipment would malfunction, and strange noises could be heard coming from the depths of the library.

Asif, Mukaffi, and Morshed soon realized that the book was more than just an ancien

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [40]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")

* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [41]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [42]:
from transformers import TrainingArguments
output_model= "Llama_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=150,
        fp16=True,
)

### **Setting up the trainer**

`max_seq_length`: Context window size


In [43]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,  # This will apply the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=train_subset,      # Pass train_subset as train_dataset
  eval_dataset=test_subset         # Pass test_subset as eval_dataset
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [44]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.758500
20,1.552800
30,1.479200
40,1.437700
50,1.357900
60,1.402800
70,1.359700
80,1.381600
90,1.355200
100,1.370600


TrainOutput(global_step=150, training_loss=1.4176239776611328, metrics={'train_runtime': 1197.7727, 'train_samples_per_second': 0.501, 'train_steps_per_second': 0.125, 'total_flos': 1.2186386694144e+16, 'train_loss': 1.4176239776611328, 'epoch': 0.56})

### **Save the model**

In [45]:
trainer.save_model("Llama_instruct_generation")

In [46]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


#### **Generate Answers After Training**

* `invoke:` call the chain on an input
* `batch:` call the chain on a list of inputs
#### **Documentation Link:**
* https://python.langchain.com/docs/expression_language/interface

# **Conversation Buffer memory**

In [47]:
# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nImagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story!\n"""
prompt = pre_prompt + "{Story}[\INST]"

prompt = PromptTemplate(template=prompt, input_variables=["Story"])

In [48]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=512, do_sample=do_sample, pad_token_id=pad_token_id, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

memory = ConversationBufferMemory()
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

#### **Example: 01**

In [49]:
context = "Once upon a time in the vibrant city of Dhaka, there were three inseparable friends - Hridoy, Mukaffi, and Wasi. Their friendship was a testament to the bond that can be formed in the most unexpected of places."
chat_llm_chain.predict(Story=context)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story!
Once upon a time in the vibrant city of Dhaka, there were three inseparable friends - Hridoy, Mukaffi, and Wasi. Their friendship was a testament to the bond that can be formed in the most unexpected of places.[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


'  Great! Here is a continuation of the story:\n\nAs the three friends navigated the bustling streets of Dhaka, they stumbled upon a mysterious old mansion that seemed to have been abandoned for decades. The mansion was hidden behind a thick wall of overgrown vegetation, and its crumbling facade seemed to whisper tales of a long-forgotten past.\n\nHridoy, being the adventurous one, suggested that they explore the mansion and uncover its secrets. Mukaffi and Wasi were hesitant at first, but their curiosity got the better of them, and they agreed to join Hridoy on his quest for discovery.\n\nAs they entered the mansion, they were struck by the eerie silence that surrounded them. The only sound was the creaking of the old wooden floorboards beneath their feet. They found themselves in a grand entrance hall, with a sweeping staircase that seemed to stretch up to the sky.\n\nSuddenly, they heard a faint whisper in the distance. It was a soft, melodic voice that seemed to be coming from ever

#### **Example: 02**

In [50]:
context = "In Dhaka's vibrant Ahsanullah University, Asif Fahim, Mukaffi, and Morshed forged a lasting bond. As students, they navigated the complexities of science and technology, each with unique dreams."
chat_llm_chain.predict(Story=context)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story!
In Dhaka's vibrant Ahsanullah University, Asif Fahim, Mukaffi, and Morshed forged a lasting bond. As students, they navigated the complexities of science and technology, each with unique dreams.[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"  Asif Fahim, Mukaffi, and Morshed were the best of friends, united by their passion for science and technology. They spent countless hours together in the university's state-of-the-art labs, experimenting with cutting-edge equipment and discussing their visions for the future. Asif, the brilliant and ambitious one, dreamed of revolutionizing the field of renewable energy, while Mukaffi, the quiet and reserved one, had a hidden talent for programming and artificial intelligence. Morshed, the charismatic and outgoing one, was a master of engineering and design, with a flair for creating innovative solutions to real-world problems.\n\nDespite their different areas of expertise, the three friends shared a common goal: to make a meaningful impact on the world through science and technology. They were determined to turn their ideas into reality, and they knew that the key to success lay in their ability to collaborate and support each other.\n\nOne day, as they were working on a project in

#### **Example: 03**

In [51]:
context = "In a happy town where everyone laughed and had big dreams, there were two siblings named Fasabbih and Mariya. Fasabbih, the little brother, always worked quietly towards his goals, while Mariya, his big sister, had a bright and energetic personality that made their small town feel alive."
chat_llm_chain.predict(Story=context)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story!
In a happy town where everyone laughed and had big dreams, there were two siblings named Fasabbih and Mariya. Fasabbih, the little brother, always worked quietly towards his goals, while Mariya, his big sister, had a bright and energetic personality that made their small town feel alive.[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"  As the two siblings grew older, their paths began to diverge. Fasabbih became increasingly fascinated with the mysterious and unknown, spending countless hours in the town's library researching ancient texts and seeking out new knowledge. Mariya, on the other hand, became more and more involved in the town's social scene, throwing herself into various charity events and community projects. Despite their different interests, the two siblings remained close, with Mariya often turning to Fasabbih for advice and support.\n\nOne day, a strange and ominous event occurred in the town. A dark cloud appeared in the sky, growing larger and more menacing by the minute. The townspeople were terrified, unsure of what to do as the cloud seemed to be bearing down on them. Fasabbih, sensing an opportunity to put his knowledge to the test, urged Mariya to join him in a desperate attempt to save their town.\n\nTogether, the two siblings embarked on a perilous journey to uncover the source of the dark

In [52]:
memory.load_memory_variables({})

{'history': 'Human: Once upon a time in the vibrant city of Dhaka, there were three inseparable friends - Hridoy, Mukaffi, and Wasi. Their friendship was a testament to the bond that can be formed in the most unexpected of places.\nAI:   Great! Here is a continuation of the story:\n\nAs the three friends navigated the bustling streets of Dhaka, they stumbled upon a mysterious old mansion that seemed to have been abandoned for decades. The mansion was hidden behind a thick wall of overgrown vegetation, and its crumbling facade seemed to whisper tales of a long-forgotten past.\n\nHridoy, being the adventurous one, suggested that they explore the mansion and uncover its secrets. Mukaffi and Wasi were hesitant at first, but their curiosity got the better of them, and they agreed to join Hridoy on his quest for discovery.\n\nAs they entered the mansion, they were struck by the eerie silence that surrounded them. The only sound was the creaking of the old wooden floorboards beneath their fee

In [53]:
print(memory.buffer)

Human: Once upon a time in the vibrant city of Dhaka, there were three inseparable friends - Hridoy, Mukaffi, and Wasi. Their friendship was a testament to the bond that can be formed in the most unexpected of places.
AI:   Great! Here is a continuation of the story:

As the three friends navigated the bustling streets of Dhaka, they stumbled upon a mysterious old mansion that seemed to have been abandoned for decades. The mansion was hidden behind a thick wall of overgrown vegetation, and its crumbling facade seemed to whisper tales of a long-forgotten past.

Hridoy, being the adventurous one, suggested that they explore the mansion and uncover its secrets. Mukaffi and Wasi were hesitant at first, but their curiosity got the better of them, and they agreed to join Hridoy on his quest for discovery.

As they entered the mansion, they were struck by the eerie silence that surrounded them. The only sound was the creaking of the old wooden floorboards beneath their feet. They found themse